In [25]:
import pandas as pd
from pprint import pprint
from datetime import datetime

In [61]:
sales_order_df = pd.read_parquet(
    "./test_data/sales_order_2025-6-9_13-11-9_744000.parquet"
)
print(sales_order_df.head(2))
# date = sales_order_df["last_updated"][0]

dim_date_df = pd.DataFrame({
    "date_id": pd.Series(dtype="int"),
    "year": pd.Series(dtype="int"),
    "month": pd.Series(dtype="int"),
    "day": pd.Series(dtype="int"),
    "day_of_week": pd.Series(dtype="int"),
    "day_name": pd.Series(dtype="str"),
    "month_name": pd.Series(dtype="str"),
    "quarter": pd.Series(dtype="int"),
})


   sales_order_id              created_at            last_updated  design_id  \
0               2 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186          3   
1               3 2022-11-03 14:20:52.188 2022-11-03 14:20:52.188          4   

   staff_id  counterparty_id  units_sold unit_price  currency_id  \
0        19                8       42972       3.94            2   
1        10                4       65839       2.91            3   

  agreed_delivery_date agreed_payment_date  agreed_delivery_location_id  
0           2022-11-07          2022-11-08                            8  
1           2022-11-06          2022-11-07                           19  


In [ ]:
def dim_date_dataframe(start_date: str, end_date: str):
    all_dates = pd.date_range(start=start_date, end=end_date)
    date_dataframe = pd.DataFrame({"date_id": all_dates})

    date_dataframe["year"] = date_dataframe["date_id"].dt.year
    date_dataframe["month"] = date_dataframe["date_id"].dt.month
    date_dataframe["day"] = date_dataframe["date_id"].dt.day
    date_dataframe["day_of_week"] = date_dataframe["date_id"].dt.weekday + 1
    date_dataframe["day_name"] = date_dataframe["date_id"].dt.day_name()
    date_dataframe["month_name"] = date_dataframe["date_id"].dt.month_name()
    date_dataframe["quarter"] = date_dataframe["date_id"].dt.quarter

    date_dataframe = date_dataframe[
        [
            "date_id",
            "year",
            "month",
            "day",
            "day_of_week",
            "day_name",
            "month_name",
            "quarter",
        ]
    ]

    return date_dataframe


In [62]:
# start_date="20100101", end_date="20351231"
# some_date = datetime.fromisoformat(dim_date["date_id"][0].isoformat())

# dim_date["date_id"].apply(lambda x: datetime.fromisoformat(x.isoformat()))
# dim_date["date_id"] = dim_date["date_id"].apply(
#     lambda x: datetime.fromtimestamp(x).strftime("%Y-%m-%d")
# )
# dim_date_converted = dim_date
# dim_date convert date_id to datetime
# print(dim_date.head(5))
# print(type(dim_date["date_id"][0]))
# pprint(datetime.now())
# pprint(some_date)
dim_date = dim_date_dataframe("20221102", "20500101")
dim_date["date_id"] = pd.to_datetime(dim_date["date_id"])
pprint(dim_date.head(2).to_dict(into={"date_id": datetime}))

{'date_id': {0: Timestamp('2022-11-02 00:00:00'),
             1: Timestamp('2022-11-03 00:00:00')},
 'day': {0: 2, 1: 3},
 'day_name': {0: 'Wednesday', 1: 'Thursday'},
 'day_of_week': {0: 3, 1: 4},
 'month': {0: 11, 1: 11},
 'month_name': {0: 'November', 1: 'November'},
 'quarter': {0: 4, 1: 4},
 'year': {0: 2022, 1: 2022}}


In [ ]:
sales_order_df["created_date"] = pd.to_datetime(
    sales_order_df["created_at"]
).dt.normalize()

sales_order_df["last_updated_date"] = pd.to_datetime(
    sales_order_df["last_updated"]
).dt.normalize()
sales_order_df["agreed_payment_date"] = pd.to_datetime(
    sales_order_df["agreed_payment_date"]
).dt.normalize()
sales_order_df["agreed_delivery_date"] = pd.to_datetime(
    sales_order_df["agreed_delivery_date"]
).dt.normalize()

pprint(sales_order_df.head(2))

   sales_order_id              created_at            last_updated  design_id  \
0               2 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186          3   
1               3 2022-11-03 14:20:52.188 2022-11-03 14:20:52.188          4   

   staff_id  counterparty_id  units_sold unit_price  currency_id  \
0        19                8       42972       3.94            2   
1        10                4       65839       2.91            3   

  agreed_delivery_date agreed_payment_date  agreed_delivery_location_id  \
0           2022-11-07          2022-11-08                            8   
1           2022-11-06          2022-11-07                           19   

  created_date       last_updated_date  
0   2022-11-03 2022-11-03 14:20:52.186  
1   2022-11-03 2022-11-03 14:20:52.188  


In [68]:
sales_order_df = sales_order_df.merge(
    dim_date[["date_id"]].rename(columns={"date_id": "created_date_id"}),
    how="left",
    left_on="created_date",
    right_on="created_date_id",
)

print(sales_order_df.head(2))

   sales_order_id              created_at            last_updated  design_id  \
0               2 2022-11-03 14:20:52.186 2022-11-03 14:20:52.186          3   
1               3 2022-11-03 14:20:52.188 2022-11-03 14:20:52.188          4   

   staff_id  counterparty_id  units_sold unit_price  currency_id  \
0        19                8       42972       3.94            2   
1        10                4       65839       2.91            3   

  agreed_delivery_date agreed_payment_date  agreed_delivery_location_id  \
0           2022-11-07          2022-11-08                            8   
1           2022-11-06          2022-11-07                           19   

  created_date       last_updated_date created_date_id_x created_date_id_y  
0   2022-11-03 2022-11-03 14:20:52.186        2022-11-03        2022-11-03  
1   2022-11-03 2022-11-03 14:20:52.188        2022-11-03        2022-11-03  
